In [42]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from nibabel.testing import data_path
import nibabel as nib
import torch.nn as nn
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode
from PIL import Image
d_r = '/home/plant99/Documents/code/pro_per/a-t-b/data/nii-data/Outputs/cpac/filt_global/png-i/'

In [43]:
num_epochs = 10
num_classes = 2
batch_size = 10
learning_rate = 0.001

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [48]:
class Rescale(object):
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size
    
    def __call__(self, sample):
        image = sample['image']
        h, w = image.shape[:2]
        
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size
        
        new_h, new_w = int(new_h), int(new_w)
        
        img = transform.resize(image, (new_h, new_w))
        return {'image': img, 'label': sample['label']}
class ToTensor(object):

    def __call__(self, sample):
        image = sample['image']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image), 'label': sample['label']}

In [49]:
class DS(Dataset):
    def __init__(self, csv_file, transform = None):
        self.transform = transforms.ToTensor()
        self.csv_ds = pd.read_csv(csv_file, dtype='str')
#         print(len(self.csv_ds))
        self.image_arr = []
        self.label_arr = []
        self.operation_arr = []
        
        for i, line in enumerate(self.csv_ds):
            self.image_arr.append(os.path.join(d_r, self.csv_ds.iloc[i]['FILE_ID']+'_alff.nii.jpg'))
            self.label_arr.append(self.csv_ds.iloc[i]['DX_GROUP'])
            self.operation_arr.append(True)
#         self.root_dir = root_dir
        print(len(self.csv_ds))
#         time.sleep(10)
    def __len__(self):
        return len(self.csv_ds)
    
    def __getitem__(self, idx):
#         print(self.csv_ds.iloc[idx]['FILE_ID'],self.csv_ds.iloc[idx]['DX_GROUP'] )
        try:
            e_f = os.path.join(d_r, self.csv_ds.iloc[idx]['FILE_ID']+'_alff.nii.jpg')
        except:
            print(self.image_arr, len(self.image_arr))
            print(idx)
        img = Image.open(e_f)
#         img = img.convert('RGB')
        
        op = True #self.operation_arr[idx]
        
        if op:
            pass
        
#         if self.csv_ds.iloc[idx]['FILE_ID'] == "no_filename":
#             e_f = os.path.join(d_r, 'Olin_0050130_alff.nii')
#         img = io.imread(e_f)
#         img_name = str(self.csv_ds.iloc[idx][0])
#         img_path = os.path.join(self.root_dir, 'train',str(self.csv_ds.iloc[idx][1]), str(self.csv_ds.iloc[idx][0]) + '.jpg')
#         image = io.imread(img_path)
        label = self.csv_ds.iloc[idx]['DX_GROUP']
#         sample = {'image': img, 'label': label}
        
        img = self.transform(img)
        
#         if self.transform:
#             sample = self.transform(sample)
        return(img, label)
#         return("asd")
    


In [69]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=2):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(51168, num_classes+1)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out
model = ConvNet(num_classes).to(device)

# # Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
# total_step = len(train_loader)

RuntimeError: cuda runtime error (59) : device-side assert triggered at /pytorch/aten/src/THC/generic/THCTensorCopy.cpp:20

In [70]:

# scale = Rescale((100, 100))
# toTensor = ToTensor()
# print(len(dl))

# for i in range(len(tds)):
    
#     sample = tds[i]
#     print(i, sample['image'].shape, sample['label'])
#     np_i = sample['image'].numpy()
#     io.imshow(np.transpose(np_i, (1, 2, 0)), interpolation='nearest')
#     tensor_image = toTensor(sample)['image']
#     scaled_image = scale(sample)['image']
#     io.imshow(scaled_image)
#     ax = plt.subplot(1, 4, i + 1)
#     plt.tight_layout()
#     ax.set_title('Sample #{}'.format(i))
#     ax.axis('off')
# #     show_landmarks(**sample)

#     if i == 2:
# #         plt.show()
#         break
#     time.sleep(2)
tds = DS(csv_file='./data/data.csv',transform = transforms.Compose([ToTensor()]) )
dl = torch.utils.data.DataLoader(dataset=tds, batch_size=batch_size, shuffle=False)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(dl):
        labels_ = []
        for label in labels:
            labels_.append(int(label)) 
        labels = torch.Tensor(labels_)
        
        print(labels, images.shape)
        images = images.to(device)
#         labels = torch.Tensor(labels)
        labels.to(device)
    
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            break
        
        

1035
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]) torch.Size([10, 3, 165, 158])


RuntimeError: cuda runtime error (59) : device-side assert triggered at /pytorch/aten/src/THC/generic/THCTensorCopy.cpp:20

In [65]:
print(device)

cuda:0
